In [11]:
import pandas as pd
import math

In [12]:
file_name = '../data/demo-4.csv'
file_name_bmms = '../data/BMMS_overview.xlsx'
file_name_roads = '../data/_roads3.csv'
demo = pd.read_csv(file_name)
bmms = pd.read_excel(file_name_bmms)
roads = pd.read_csv(file_name_roads)

Roads required: N1 N105 N102 N120 N104 N2 N204 N207 N208

In [21]:
def clean_file(roads, bmms, road_name):
    # Filter the 'roads' DataFrame for rows where the 'road' column is 'N1'
    n1_roads = roads[roads['road'] == road_name]
    n1_roads = n1_roads[(n1_roads['lon'] <= 91.851) &
                     (n1_roads['lat'] >= 22.36)]
    
    # Generate an ID sequence starting from 1
    n1_roads['id'] = range(1, len(n1_roads) + 1)
    
    # Set model_type to 'link' for all rows
    n1_roads['model_type'] = 'link'
    
    # Generate 'name' as "Link" + id as string
    #n1_roads['name'] = ['Link ' + str(id) for id in n1_roads['id']]
    n1_roads['name'] = 'Link'
    # Calculate 'length' as difference between this row's 'chainage' and the next row's 'chainage'
    # Shift(-1) moves the chainage up by one row to subtract, fillna(0) to handle the last item
    n1_roads['length'] = (n1_roads['chainage'].shift(-1) - n1_roads['chainage']).fillna(0)
    
    # Selecting the columns needed for the empty DataFrame
    n1_roads_final = n1_roads[['road', 'id', 'model_type', 'name', 'lat', 'lon', 'length', 'chainage']]
    
    # Filter BMMS data for road 'N1'
    bmms_n1 = bmms[bmms['road'] == road_name].copy()
    bmms_n1 = bmms_n1[(bmms_n1['lon'] <= 91.851) &
                     (bmms_n1['lat'] >= 22.363)]
    
    bmms_n1 = bmms_n1[~bmms_n1['name'].str.contains(r"\(R\)", na=False)]
    bmms_n1 = bmms_n1[~bmms_n1['name'].str.contains("right", case=False, na=False)]
    bmms_n1 = bmms_n1[~bmms_n1['name'].str.contains("RIGHT", case=False, na=False)]
    bmms_n1 = bmms_n1[~bmms_n1['name'].str.contains(r"\( R \)", na=False)]
    
    # Set up for new entries
    bmms_n1['model_type'] = 'bridge'
    #bmms_n1['name'] = ['Bridge ' + str(i+1) for i in range(bmms_n1.shape[0])]
    bmms_n1['id'] = range(n1_roads_final['id'].max() + 1, n1_roads_final['id'].max() + 1 + bmms_n1.shape[0])
    bmms_n1['chainage'] = bmms_n1['km']  # Use 'km' as 'chainage'
    bmms_n1['length'] = bmms_n1['length'] / 1000
    
    # Select and rename columns to match the format of n1_roads_final_with_chainage
    bmms_n1_formatted = bmms_n1[['road', 'id', 'model_type', 'name', 'lat', 'lon', 'chainage', 'length', 'condition']]
    
    # Combine the dataframes and sort by chainage
    combined_df = pd.concat([n1_roads_final, bmms_n1_formatted], ignore_index=True).sort_values(by='chainage')
    
    combined_df.iloc[0, 2] = 'source'
    combined_df.iloc[-1, combined_df.columns.get_loc('model_type')] = 'sink'
    combined_df.reset_index(drop=True, inplace=True)
    count = 1
    for index, row in combined_df.iterrows():
        combined_df.iloc[index, 1] = count
        count += 1
    
    n1 = combined_df
    
    duplicates_df = bmms_n1[bmms_n1.duplicated('km', keep=False)]

    # Assuming duplicates_df is your DataFrame


    # Convert 'condition' to a numerical value for averaging
    condition_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}
    n1['condition_num'] = n1['condition'].map(condition_mapping)
    
    #add together the links
    amount = 0
    condition_sum = 0
    rows_to_add = []  # List to accumulate rows
    last_row = None
    
    # Initialize the DataFrame
    n1_new = pd.DataFrame(columns=['road', "id", 'model_type', 'name', 'lon', 'lat', 'length', 'condition', 'condition_num'])
    for index, row in n1.iterrows():
        if last_row is not None:
            if row['model_type'] == 'bridge':
                amount += 1
                condition_sum += row['condition_num']
            if row['model_type'] != 'bridge' and last_row['model_type'] != 'bridge':
                rows_to_add.append({'road': row['road'], 'id': row['id'], 'model_type': row['model_type'],
                                    'name': row['name'],'lon': row['lon'], 'lat':row['lat'], 'length': row['length'], 'condition': row['condition'],
                                    'condition_num': row['condition_num']})
                amount = 0
                condition_sum = 0
            if row['model_type'] != 'bridge' and last_row['model_type'] == 'bridge':
                rows_to_add.append({'road': last_row['road'], 'id': last_row['id'], 'model_type': last_row['model_type'],
                                    'name': last_row['name'],'lon': row['lon'], 'lat':row['lat'], 'length': last_row['length'],
                                    'condition': last_row['condition'], 'condition_num': condition_sum / amount})
                amount = 0
                condition_sum = 0
                rows_to_add.append({'road': row['road'], 'id': row['id'], 'model_type': row['model_type'],
                                    'name': row['name'],'lon': row['lon'], 'lat':row['lat'], 'length': row['length'], 'condition': row['condition'],
                                    'condition_num': row['condition_num']})
        else:
            rows_to_add.append({'road': row['road'], 'id': row['id'], 'model_type': row['model_type'],
                                'name': row['name'],'lon': row['lon'], 'lat':row['lat'], 'length': row['length'], 'condition': row['condition'],
                                'condition_num': row['condition_num']})
        last_row = row
    
    n1_new = pd.concat([n1_new, pd.DataFrame(rows_to_add)], ignore_index=True)

    for index, row in n1_new.iterrows():
        if pd.isna(row['condition_num']) == False:
            n1_new.iloc[index, 8] = math.ceil(n1_new.iloc[index, 8])
    condition_mapping = {1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E'}
    n1_new['condition'] = n1_new['condition_num'].map(condition_mapping)
    # n1_new
    # n1_new['id'] = range(1, len(n1_new) + 1)
    # for index, row in n1_new.iterrows():
    #     if row['model_type'] != 'bridge':
    #         n1_new.iloc[index, 3] = row['model_type'] + ' ' + str(row['id'])
    #     else:
    #         n1_new.iloc[index, 3] = n1_new.iloc[index, 3] + ' ' + str(row['id'])
    # print(n1_new)
    return n1_new

In [27]:
all_roads = []
for road in ['N1', 'N105', 'N102', 'N120', 'N104', 'N2', 'N204', 'N207', 'N208']:
    all_roads.append(clean_file(roads, bmms, road))
merged_dataframe = pd.concat(all_roads, ignore_index=True)
desired_order = ['road', 'id', 'model_type', 'condition', 'name', 'lat', 'lon', 'length']
merged_dataframe = merged_dataframe[desired_order]
count = 1
for index, row in merged_dataframe.iterrows():
    merged_dataframe.loc[index, 'id'] = str(count)
    count+=1
    if row.model_type == 'link':
        merged_dataframe.loc[index, 'name'] = 'link ' + merged_dataframe.loc[index, 'id']
    elif row.model_type == 'source':
        merged_dataframe.loc[index, 'name'] = 'source ' + merged_dataframe.loc[index, 'id']
    elif row.model_type == 'sink':
        merged_dataframe.loc[index, 'name'] = 'sink ' + merged_dataframe.loc[index, 'id']
    elif row.model_type == 'bridge':
        merged_dataframe.loc[index, 'name'] = merged_dataframe.loc[index, 'name'] + ' id ' + merged_dataframe.loc[index, 'id']


C:\Users\svenk\AppData\Local\Temp\ipykernel_14100\598167041.py:99: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  n1_new = pd.concat([n1_new, pd.DataFrame(rows_to_add)], ignore_index=True)
C:\Users\svenk\AppData\Local\Temp\ipykernel_14100\598167041.py:99: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  n1_new = pd.concat([n1_new, pd.DataFrame(rows_to_add)], ignore_index=True)
C:\Users\svenk\AppData\Local\Temp\ipykernel_14100\598167041.py:99: FutureWarning: The behavior of DataFrame concatenation wi

In [28]:
merged_dataframe

,road,id,model_type,condition,name,lat,lon,length
0,N1,1,source,NaN,source 1,23.706028,90.443333,0.8140
1,N1,2,link,NaN,link 2,23.702917,90.450417,0.0080
2,N1,3,link,NaN,link 3,23.702778,90.450472,0.1780
3,N1,4,link,NaN,link 4,23.702139,90.451972,1.0000
4,N1,5,bridge,A,. id 5,23.697889,90.460583,0.0113
...,...,...,...,...,...,...,...,...
2876,N208,2877,bridge,A,Chatura Box Culvert id 2877,24.513472,91.843194,0.0016
2877,N208,2878,link,NaN,link 2878,24.513472,91.843194,0.1620
2878,N208,2879,link,NaN,link 2879,24.514250,91.844472,0.3660
2879,N208,2880,bridge,A,GOBINDO PATI BOX CULVERT id 2880,24.516028,91.847500,0.0124


In [29]:
merged_dataframe.to_csv('../data/merged_data.csv', index=False)